# A fast quantum mechanical algorithm for database search

Classical Algorithms - Minimum $\frac{N}{2}$ lookups.

Quantum mechanical systems can be in a superposition of states and simultaneously examine multiple names. $O(\sqrt{N})$ steps.

Note - This is the problem of integer factorization, i.e. finding the factors of a given integer N, in a time which is polynomial in $\log{N}$.

The problem is this: there is an unsorted database containing N items out of which just one item satisfies a given condition - that one item has to be retrieved.
Quantum mechanical systems can simultaneously be in multiple Schrodinger cat states and carry out multiple tasks at the same time.

In these algorithms, instead of having the system in a specified state, it is in a distribution over various states with a certain probability of being in each state. At each step, there is a certain probability of making a transition from one state to another. The evolution of the system is obtained by premultiplying this probability vector (that describes the distribution of probabilities over various states) by a state transition matrix. Knowing the initial distribution and the state transition matrix, it is possible in principle to calculate the distribution at any instant in time.

The machinery of quantum mechanical algorithms is illustrated by discussing the three operations that are needed in the algorithm of this paper. The first is the creation of a configuration in which the amplitude of the system being in any of the $2^n$ basic states of the system is equal; the second is the Walsh-Hadamard transformation operation and the third the selective rotation of different states.

Problem Statement - Let a system have $N = 2^n$ states which are labelled $S_1, S_2, \dots S_N$. These $2^n$ states are represented as $n$ bit strings. Let there be a unique state, say $S_v$, that satisfies the condition $C(S_v) = 1$, whereas for all other states S, $C(S) = 0$ (assume that for any state S, the condition C(S) can be evaluated in unit time). The problem is to identify the state $S_v$.

Algorithm -
Initialize the system to the distribution: $(\frac{1}{\sqrt{N}}, \frac{1}{\sqrt{N}}, \frac{1}{\sqrt{N}}, \dots, \frac{1}{\sqrt{N}})$, i.e. there is the same amplitude to be in each of the N states.

Repeat the following unitary operations $O(\sqrt{N})$ times:
1. Let the system be in any state S: In case $C(S) = 1$, rotate the phase by $\pi$ radians; In case $C(S) = 0$, leave the system unaltered.
2. Apply the diffusion transform D which is defined by the matrix D as follows: $ D_{ij} = \frac{2}{N}; i \neq j$ & $D_{ii} = -1 + \frac{2}{N}$. This diffusion transform, D, can be implemented as $D = WRW$, where R the rotation matrix & W the Walsh-Hadamard Transform Matrix are defined as follows:
$R_{ij} = 0; i \neq j$ & $R_{ii} = 1; i = 0$ & $R_{ii} = -1; i \neq 0$
3. Sample the resulting state. In case $C(S_v) = 1$ there is a unique state $S_v$ such that the final state is $S_v$ with a probability of at least $\frac{1}{2}$.

The implementation does not involve a classical measurement.

The loop in step (2) above, is the heart of the algorithm. Each iteration of this loop increases the amplitude in the desired state by $O(\frac{1}{\sqrt{N}})$ as a result in $O(\sqrt{N})$ repetitions of the loop, the amplitude and hence the probability in the desired state reach $O(1)$.

The diffusion transform, D, can be interpreted as an inversion about average operation.

The only operations required are, first, the Walsh-Hadamard transform, and second, the conditional phase shift operation both of which are relatively easy as compared to operations required for other quantum mechanical algorithms.

Reference:

Grover, L. K. (1996, July). A fast quantum mechanical algorithm for database search. In Proceedings of the twenty-eighth annual ACM symposium on Theory of computing (pp. 212-219) - https://arxiv.org/abs/quant-ph/9605043

In [5]:
import numpy as np
from qiskit.quantum_info import Operator

n = 3

D = np.zeros(shape=(2 ** n, 2 ** n))

for i in range(2 ** n):
    for j in range(2 ** n):
        if i != j:
            D[i, j] = 2 / (2 ** n)
        elif i == j:
            D[i, j] = -1 + (2 / (2 ** n))

D = Operator(D)
D.draw("latex")

<IPython.core.display.Latex object>

In [6]:
from qiskit.quantum_info import Statevector

psi_input = Statevector((1 / np.sqrt(8)) * np.array([1, -1, 1, 1, 1, 1, -1, 1]))
psi_input.draw("latex")

<IPython.core.display.Latex object>

In [9]:
psi_input.evolve(D).draw("latex")

<IPython.core.display.Latex object>

In [25]:
from qiskit.circuit.library import HGate

H = Operator(HGate()).tensor(Operator(HGate()))
H.draw("latex")

<IPython.core.display.Latex object>

In [26]:
R = np.zeros(shape=(2 ** n, 2 ** n))

for i in range(2 ** n):
    for j in range(2 ** n):
        if i != j:
            R[i, j] = 0
        elif i == j == 0:
            R[i, j] = 1
        elif i == j != 0:
            R[i, j] = -1

new_D = H @ R @ H

new_D = Operator(new_D)
new_D.draw("latex")

<IPython.core.display.Latex object>

In [27]:
D == new_D

True

# Grover Adaptive Search (GAS) for Constrained Polynomial Binary Optimization (CPBO)

GAS can provide a quadratic speed-up for combinatorial optimization problems com- pared to brute force search. A commonly-studied class of combinatorial optimization problems are Quantum Unconstrained Binary Optimization (QUBO) problems, with applications in resource allocation, finance, machine learning, and partitioning. GAS iteratively applies Grover Search to find the optimum value of an objective function, by us- ing the best-known value as a threshold to flag all values smaller than the threshold in order to find a better solution.

The objectivefunction and constraints need to be efficiently encoded, for which we use a Quantum Dictionary, a pattern for representing key-value pairs as entangled quantum registers, that turns out to be efficientfor polynomial functions – in particular for quadratics representing QUBO problems.

Grover Search is often described as a search algorithm, because it was initially formulated in the context of finding a single state of interest in a superposition of n-qubit quantum states. The algorithm has been generalized to the case of multiple states of interest, in which case it is better interpreted as a sampling algorithm. It amplifies the amplitudes of the states of interest within a larger search space, thus, increasing the probability of measuring one of the target states.

Grover Search – the core element of GAS – needs three ingredients:
1. A state preparation operator A to construct a superposition of all states in the search space. In this manuscript, A is implemented by Hadamard gates $H^{\otimes n}$, i.e. it constructs the equal superposition state:
$$ H^{\otimes n} |0\rangle = \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^n - 1} |i\rangle_n $$

2. An oracle operator O, that recognizes the states of interest and multiplies their amplitudes by $-1$. For instance, suppose $I \in \{0, \dots, 2^n − 1\}$ denotes the set of target states and $A = H^{\otimes n}$, then
$$ OA|0\rangle_n = \frac{1}{\sqrt{2^n}} \sum_{i \notin I} |i\rangle_n - \frac{1}{\sqrt{2^n}} \sum_{i \in I} |i\rangle_n $$

3. The Grover diffusion operator D, that multiplies the amplitude of the $|0\rangle_n$ state (or, equivalently, all states except $|0\rangle_n$) by -1. The diffusion operator has the net effect of inverting all amplitudes in the quantum state about their mean. This causes all the amplitudes of the states of interest to be magnified, while the amplitudes of all other states are decreased.

Consider a function $f : X \rarr R$ for n binary variables, where for ease of presentation assume $X = {0, 1}^n$, for which we are interested in finding $min_{x \in X} f(x)$. The main idea of GAS is to construct $A_y$ and $O_y$ for a given threshold y such that they flag all states $x \in X$ satisfying $f(x) < y$, such that we can use Grover Search to find a solution $\tilde{x}$ with a function value better than y. Then we set $y = f(\tilde{x})$ and repeat until some formal termination criteria is met, e.g. based on the number of iterations, time, or progress in y.

QUBO and CCPBO Oracle Problem: $f(x) = x^TQx + b^Tx + c$

We can add additional registers to evaluate other polynomials. Whether an inequality constraint is satisfied or not can again be mapped to the sign qubit by applying an appropriate shift to the polynomial. Equality constraints are a bit more expensive, as they require the detection of a particular state, which essentially has the same complexity as the Grover diffusion operator D. This leads to a set of qubits flagging target states: one qubit identifying the states that correspond to objective values below the current threshold, and one qubit for each constraint.

Reference:

Gilliam, A., Woerner, S., & Gonciulea, C. (2021). Grover adaptive search for constrained polynomial binary optimization. Quantum, 5, 428 - https://arxiv.org/abs/1912.04088

# Quantum Amplitude Amplification and Estimation

